# SIB KAGGLE PROJECT

In [1]:
#Imports
import pprint
from utils.func import *
import pandas as pd
import numpy as np
from sklearn import preprocessing
from utils.models import SupervisedModelOptimization

In [2]:
# First lets see the data!
# Train dataframeW
data = pd.read_csv("data/train.csv")
# Test dataframe
validation = pd.read_csv("data/test.csv")
print(f"Data is divided in {data.shape[0]} lines and {data.shape[1]} col")
print(f"Validation data is divided in {validation.shape[0]} lines and {validation.shape[1]} col")
print(f"Labels: {[labels for labels in data.columns]}")
print("Remove data_source and seq_id")
data = data.drop(columns=["seq_id", "data_source"])
validation = validation.drop(columns=["seq_id", "data_source"])
print(f"Data is divided in {data.shape[0]} lines and {data.shape[1]} col")
print(f"Validation data is divided in {validation.shape[0]} lines and {validation.shape[1]} col")
print("We want to predict tm values for test data")


Data is divided in 31390 lines and 5 col
Validation data is divided in 2413 lines and 4 col
Labels: ['seq_id', 'protein_sequence', 'pH', 'data_source', 'tm']
Remove data_source and seq_id
Data is divided in 31390 lines and 3 col
Validation data is divided in 2413 lines and 2 col
We want to predict tm values for test data


### The data possesses swapped values between the pH and tm columns, then, "swap_ph_tm" was built to fix this issue.

In [3]:
update_train = pd.read_csv("data/train_updates_20220929.csv")
data = swap_ph_tm(data, update_train)

In [4]:
data

,protein_sequence,pH,tm
0,AAAAKAAALALLGEAPEVVDIWLPAGWRQPFRVFRLERKGDGVLVG...,NaN,NaN
1,AAADGEPLHNEEERAGAGQVGRSLPQESEEQRTGSRPRRRRDLGSR...,NaN,NaN
2,AAAFSTPRATSYRILSSAGSGSTRADAPQVRRLHTTRDLLAKDYYA...,NaN,NaN
3,AAASGLRTAIPAQPLRHLLQPAPRPCLRPFGLLSVRAGSARRSGLL...,NaN,NaN
4,AAATKSGPRRQSQGASVRTFTPFYFLVEPVDTLSVRGSSVILNCSA...,NaN,NaN
...,...,...,...
31385,YYMYSGGGSALAAGGGGAGRKGDWNDIDSIKKKDLHHSRGDEKAQG...,7.0,51.8
31386,YYNDQHRLSSYSVETAMFLSWERAIVKPGAMFKKAVIGFNCNVDLI...,7.0,37.2
31387,YYQRTLGAELLYKISFGEMPKSAQDSAENCPSGMQFPDTAIAHANV...,7.0,64.6
31388,YYSFSDNITTVFLSRQAIDDDHSLSLGTISDVVESENGVVAADDAR...,7.0,50.7


In [5]:
print(data.isnull().sum().sort_values(ascending=False))
print()
print(validation.isnull().sum().sort_values(ascending=False))
# There are some missing values in train
# Data_source values are not that important


pH                  2694
tm                  2409
protein_sequence       0
dtype: int64

protein_sequence    0
pH                  0
dtype: int64


In [6]:
missing_data = data[data["pH"].isnull()]
missing_data


,protein_sequence,pH,tm
0,AAAAKAAALALLGEAPEVVDIWLPAGWRQPFRVFRLERKGDGVLVG...,NaN,NaN
1,AAADGEPLHNEEERAGAGQVGRSLPQESEEQRTGSRPRRRRDLGSR...,NaN,NaN
2,AAAFSTPRATSYRILSSAGSGSTRADAPQVRRLHTTRDLLAKDYYA...,NaN,NaN
3,AAASGLRTAIPAQPLRHLLQPAPRPCLRPFGLLSVRAGSARRSGLL...,NaN,NaN
4,AAATKSGPRRQSQGASVRTFTPFYFLVEPVDTLSVRGSSVILNCSA...,NaN,NaN
...,...,...,...
28753,MVLKQRANYLGFLIVFFTAFLVEAVPIKRQSNSTVDSLPPLIPSRT...,NaN,58.9
28754,MVLKQRANYLGFLIVFFTAFLVEAVPIKRQSNSTVDSLPPLIPSRT...,NaN,59.4
28755,MVLKQRANYLGFLIVFFTAFLVEAVPIKRQSNSTVDSLPPLIPSRT...,NaN,57.8
28756,MVLKQRANYLGFLIVFFTAFLVEAVPIKRQSNSTVDSLPPLIPSRT...,NaN,59.3


In [7]:
data = data.drop((missing_data).index).reset_index(drop=True)
data
# Podemos remover também a data_source? Não deve de trazer nada de relevante para a analise dos dados


,protein_sequence,pH,tm
0,AAPDEITTAWPVNVGPLNPHLYTPNQMFAQSMVYEPLVKYQADGSV...,7.0,48.4
1,AARRFSGPRNQRQQGGGDPGLMHGKTVLITGANSGLGRATAAELLR...,7.0,48.4
2,AASSPEADFVKKTISSHKIVIFSKSYCPYCKKAKSVFRELDQVPYV...,7.0,49.0
3,AATFAYSQSQKRSSSSPGGGSNHGWNNWGKAAALASTTPLVHVASV...,5.5,55.6
4,AAVLVTFIGGLYFITHHKKEESETLQSQKVTGNGLPPKPEERWRYI...,7.0,48.4
...,...,...,...
28691,YYMYSGGGSALAAGGGGAGRKGDWNDIDSIKKKDLHHSRGDEKAQG...,7.0,51.8
28692,YYNDQHRLSSYSVETAMFLSWERAIVKPGAMFKKAVIGFNCNVDLI...,7.0,37.2
28693,YYQRTLGAELLYKISFGEMPKSAQDSAENCPSGMQFPDTAIAHANV...,7.0,64.6
28694,YYSFSDNITTVFLSRQAIDDDHSLSLGTISDVVESENGVVAADDAR...,7.0,50.7


In [8]:
print(f"Train data is divided in {data.shape[0]} lines and {data.shape[1]} col")
print(f"Validation data is divided in {validation.shape[0]} lines and {validation.shape[1]} col")
print(f"Labels: {[labels for labels in data.columns]}")
# 28696 


Train data is divided in 28696 lines and 3 col
Validation data is divided in 2413 lines and 2 col
Labels: ['protein_sequence', 'pH', 'tm']


In [9]:
data.describe()

,pH,tm
count,28696.000000,28696.000000
mean,6.852437,49.079321
std,1.168815,14.210971
min,1.990000,-1.000000
25%,7.000000,41.900000
50%,7.000000,48.000000
75%,7.000000,53.800000
max,64.900000,130.000000


In [10]:
data_array = np.array(data.loc[:, ["protein_sequence"]])
# test_array = np.array(test.loc[:, ["protein_sequence"]])

data_array

array([['AAPDEITTAWPVNVGPLNPHLYTPNQMFAQSMVYEPLVKYQADGSVIPWLAKSWTHSEDGKTWTFTLRDDVKFSNGEPFDAEAAAENFRAVLDNRQRHAWLELANQIVDVKALSKTELQITLKSAYYPFLQELALPRPFRFIAPSQFKNHETMNGIKAPIGTGPWILQESKLNQYDVFVRNENYWGEKPAIKKITFNVIPDPTTRAVAFETGDIDLLYGNEGLLPLDTFARFSQNPAYHTQLSQPIETVMLALNTAKAPTNELAVREALNYAVNKKSLIDNALYGTQQVADTLFAPSVPYANLGLKPSQYDPQKAKALLEKAGWTLPAGKDIREKNGQPLRIELSFIGTDALSKSMAEIIQADMRQIGADVSLIGEEESSIYARQRDGRFGMIFHRTWGAPYDPHAFLSSMRVPSHADFQAQQGLADKPLIDKEIGEVLATHDETQRQALYRDILTRLHDEAVYLPISYISMMVVSKPELGNIPYAPIATEIPFEQIKPVK'],
       ['AARRFSGPRNQRQQGGGDPGLMHGKTVLITGANSGLGRATAAELLRLGARVIMGCRDRARAEEAAGQLRQELCQAGGAGPDGTDGQLVVKELDLASLRSVRAFCQELLQEEPRLDVLINNAGVFHCPYTKTEDGFEMQFGVNHLGHFLLTNLLLGLLKSSAPSRIVVVSSKLYKYGEINFEDLNSEQSYNKSFCYSRSKLANILFTRELARRLEGTNVTVNVLHPGIVRTNLGRHIHIPLLARPLFNLVSWAFFKTPLEGAQTSIYLACSPDVEGVSGRYFGDCKEEELLPKAMDESVARKLWDISEVMVGIL'],
       ['AASSPEADFVKKTISSHKIVIFSKSYCPYCKKAKSVFRELDQVPYVVELDEREDGWSIQTALGEIVGRRTVPQVFINGKHLGGSDDTVDAYESGELAKLLGVSGNKEAE'],
       ...,
       ['YYQRTLGAELLYKISFG

In [11]:
# Data dipeptide
data_dipeptide = calculate_dipeptide_composition(data_array)
data_dipeptide_df = pd.DataFrame(data_dipeptide)

# Test dipeptide
# test_dipeptide = calculate_dipeptide_composition(test_array)
# test_dipeptide_df = pd.DataFrame(test_dipeptide)
# test_dipeptide_df


### Add values to Train data

In [12]:
molecular_weight = calculate_molecular_weight(data_array)
isoelectric_point = calculate_isoelectric_point(data_array)
aromaticity = calculate_aromaticity(data_array)
instability_index = calculate_instability_index(data_array)

data["molecular_weight"] = molecular_weight
data["isoelectric_point"] = isoelectric_point
data["aromaticity"] = aromaticity
data["instability_index"] = instability_index

# Merge the dipeptide features
data = data.join(data_dipeptide_df)
data

,protein_sequence,pH,tm,molecular_weight,isoelectric_point,aromaticity,instability_index,AA,AR,AN,...,VL,VK,VM,VF,VP,VS,VT,VW,VY,VV
0,AAPDEITTAWPVNVGPLNPHLYTPNQMFAQSMVYEPLVKYQADGSV...,7.0,48.4,56204.2095,5.507664,0.097804,31.797804,0.40,0.40,0.40,...,0.40,0.80,0.20,0.20,0.40,0.40,0.00,0.00,0.40,0.20
1,AARRFSGPRNQRQQGGGDPGLMHGKTVLITGANSGLGRATAAELLR...,7.0,48.4,34386.0086,8.195366,0.070288,41.705431,0.96,1.92,0.64,...,0.96,0.32,0.32,0.32,0.00,0.96,0.32,0.00,0.00,0.64
2,AASSPEADFVKKTISSHKIVIFSKSYCPYCKKAKSVFRELDQVPYV...,7.0,49.0,11936.2894,5.417575,0.082569,28.518349,0.93,0.00,0.00,...,0.00,0.93,0.00,1.85,1.85,0.93,0.00,0.00,0.00,0.93
3,AATFAYSQSQKRSSSSPGGGSNHGWNNWGKAAALASTTPLVHVASV...,5.5,55.6,37040.5511,5.361247,0.130699,27.470243,0.91,0.00,0.91,...,0.30,0.30,0.00,0.30,0.00,0.00,0.30,0.00,0.30,0.30
4,AAVLVTFIGGLYFITHHKKEESETLQSQKVTGNGLPPKPEERWRYI...,7.0,48.4,31215.4943,9.511939,0.046763,67.340288,1.81,0.36,0.00,...,0.36,0.72,0.00,0.00,0.36,0.00,0.72,0.00,0.00,0.36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28691,YYMYSGGGSALAAGGGGAGRKGDWNDIDSIKKKDLHHSRGDEKAQG...,7.0,51.8,61997.6230,8.466327,0.089253,48.604026,0.73,0.73,0.00,...,0.55,0.18,0.18,0.55,0.55,0.36,0.00,0.55,0.18,0.91
28692,YYNDQHRLSSYSVETAMFLSWERAIVKPGAMFKKAVIGFNCNVDLI...,7.0,37.2,52637.6897,6.532244,0.087420,35.285096,1.07,0.21,0.85,...,0.64,0.43,0.21,0.21,0.21,0.43,0.00,0.00,0.21,0.85
28693,YYQRTLGAELLYKISFGEMPKSAQDSAENCPSGMQFPDTAIAHANV...,7.0,64.6,14203.9225,5.271214,0.117188,44.198437,0.79,0.00,1.57,...,0.79,0.79,0.00,0.00,0.79,0.00,0.79,0.00,0.00,0.79
28694,YYSFSDNITTVFLSRQAIDDDHSLSLGTISDVVESENGVVAADDAR...,7.0,50.7,64367.8724,5.688128,0.074199,36.618398,1.01,0.68,0.00,...,1.01,0.34,0.34,0.51,0.17,0.84,0.34,0.00,0.17,1.01


### Add values to Test data

In [13]:
# molecular_weight = calculate_molecular_weight(test_array)
# isoelectric_point = calculate_isoelectric_point(test_array)
# aromaticity = calculate_aromaticity(test_array)
# instability_index = calculate_instability_index(test_array)

# test["molecular_weight"] = molecular_weight
# test["isoelectric_point"] = isoelectric_point
# test["aromaticity"] = aromaticity
# test["instability_index"] = instability_index

# # Merge the dipeptide features
# test = test.join(test_dipeptide_df)
# test

# Supervised Machine Learning

## Pre-Processing

#### Verificação da Média e Desvio Padrão dos dados de treino:

In [14]:
data_input = data.loc[:, ~data.columns.isin(["protein_sequence", "tm"])].values
data_output = data.loc[:, "tm"].values

pre_processed_data = preprocessing.scale(data_input)

print("Media global: ", pre_processed_data.mean())
print("Desvio padrao global: ", pre_processed_data.std())
print("\nVerificar se a média e o desvio padrão estão próximos dos valores 0 e 1, respetivamente.")
print("\tMédia:", ((pre_processed_data.mean(axis=0) < 0.000001) & (pre_processed_data.mean(axis=0) > -0.000001)).all())
print("\tDesvio Padrão:", ((pre_processed_data.std(axis=0) < 1.000001) & (pre_processed_data.std(axis=0) > 0.999999)).all())

Media global:  -3.815033945577674e-18
Desvio padrao global:  1.0

Verificar se a média e o desvio padrão estão próximos dos valores 0 e 1, respetivamente.
	Média: True
	Desvio Padrão: True


### Divide the train and test data

In [15]:
num_test = int(pre_processed_data.shape[0] * 0.3)

print("Número de exemplos para teste:", num_test)

indices = np.random.permutation(len(data_input))

# Get the Input data pre-processed according with the indexes
train_input = pre_processed_data[indices[:-num_test]]
test_input = pre_processed_data[indices[-num_test:]]

# Get the output data according with the indexes
train_output = data_output[indices[:-num_test]]
test_output = data_output[indices[-num_test:]]

Número de exemplos para teste: 8608


In [16]:
# verificar as dimensões
print(train_input.shape)
print(train_output.shape)

print(test_input.shape)
print(test_output.shape)

(20088, 405)
(20088,)
(8608, 405)
(8608,)


## KNRegressor

In [17]:
from sklearn.neighbors import KNeighborsRegressor

knn = KNeighborsRegressor(3)
knn.fit(train_input, train_output)
test_prevision = knn.predict(test_input)

print(knn.score(test_input, test_output))


0.5080554676582711


### Error evaluation
Check: https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics

In [18]:
calculate_model_error(test_output, test_prevision)


Root Maximum Residual Error: 8.698
Root Mean Absolute Error: 2.635
Root Mean Square Error: 10.102


## Decision Tree Regressor

In [45]:
from sklearn.tree import DecisionTreeRegressor

tree_model = DecisionTreeRegressor()
tree_model = tree_model.fit(train_input, train_output)
tree_prevision = tree_model.predict(test_input)

print(tree_model.score(test_input, test_output))

0.08400460835007517


In [20]:
calculate_model_error(test_output, tree_prevision)

Root Maximum Residual Error: 8.456
Root Mean Absolute Error: 3.042
Root Mean Square Error: 13.782


## Naive Bayes
### SHOULD NOT BE USED WITH NEGATIVE VALUES -> VERIFY THIS

In [22]:
# from sklearn.naive_bayes import CategoricalNB

# cnb_model = CategoricalNB()
# cnb_model = cnb_model.fit(train_input, train_output)
# cnb_prevision = cnb_model.predict(test_input)

# print(cnb_model.score(test_input, test_output))

In [23]:
# calculate_model_error(test_output, cnb_prevision)

## Linear Regression

In [40]:
from sklearn import linear_model

lregr_model = linear_model.LinearRegression()
lregr_model = lregr_model.fit(train_input, train_output)

lregr_prevision = lregr_model.predict(test_input)

print("Comparison between the 5 first predicted and real output values:")
lregr_comparison = list(zip(lregr_prevision, test_output))[:5]
print(" | ".join(f"{round(values[0], 1)}, {values[1]}" for values in lregr_comparison))

Comparison between the 5 first predicted and real output values:
0.9, 25.0 | 12.2, 44.7 | -14.9, 22.0 | 4.5, 38.3 | -8.3, 42.6


## Support Vector Machines

In [43]:
from sklearn.svm import SVR
from sklearn.metrics import accuracy_score

# Create an SVM model with a linear kernel
svm = SVR(kernel='linear')

# Fit the model to the training data
svm.fit(train_input, train_output.astype("int32"))

# Make predictions on the test data
y_pred = svm.predict(test_input)

# Previsions
print(y_pred)

# Accuracy
calculate_model_error(test_output, y_pred)

[54.9770263  56.80112715 36.4443912  ... 50.47336431 54.4371739
 51.00308501]
Root Maximum Residual Error: 11.885
Root Mean Absolute Error: 2.897
Root Mean Square Error: 11.933


## Random Forest Regressor

In [44]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=100, max_depth=5, random_state=42)

# Train the model
rf.fit(train_input, train_output)

# Evaluate the model on the test set
y_pred = rf.predict(test_input)
rf_comparison = list(zip(y_pred, test_output))[:10]
print(" | ".join(f"{round(values[0], 1)}, {values[1]}" for values in rf_comparison))

# Accuracy
calculate_model_error(test_output, y_pred)

48.2, 25.0 | 48.5, 44.7 | 45.7, 22.0 | 51.6, 38.3 | 48.1, 42.6 | 56.4, 51.6 | 48.5, 38.7 | 48.1, 49.6 | 48.1, 47.5 | 48.1, 44.7
Root Maximum Residual Error: 8.940
Root Mean Absolute Error: 2.961
Root Mean Square Error: 12.186


In [47]:
rf.score(test_input, test_output)

0.2841615095831529

## Bagging Regressor

In [46]:
from sklearn.ensemble import BaggingRegressor

# Define the base regressor
base_regressor = DecisionTreeRegressor(random_state=42)

# Define the Bagging regressor
bag_reg = BaggingRegressor(base_estimator=base_regressor,
                            n_estimators=100,
                            random_state=42)

# Train the model
bag_reg.fit(train_input, train_output)

# Evaluate the model on the test set
y_pred = bag_reg.predict(test_input)
bag_reg_comparison = list(zip(y_pred, test_output))[:10]
print(" | ".join(f"{round(values[0], 1)}, {values[1]}" for values in bag_reg_comparison))

# Accuracy
calculate_model_error(test_output, y_pred)

63.5, 25.0 | 57.0, 44.7 | 22.0, 22.0 | 49.1, 38.3 | 39.2, 42.6 | 55.7, 51.6 | 48.8, 38.7 | 47.1, 49.6 | 51.5, 47.5 | 44.4, 44.7
Root Maximum Residual Error: 7.876
Root Mean Absolute Error: 2.625
Root Mean Square Error: 10.076


# Supervised Machine Learning V2

Supervised Machine Learning has the distinct characteristic of being trained on a labeled dataset. Hereon, the same training dataset of the unsupervised ML will be used with the correct output labeled.

The algorithm, through the recognition of patterns in the training data, will provide predictions in new unseen data. 

SupervisedModelOptimization class was built under the utils/models.py file and uses several useful Supervised Machine Learning models in regression problems, which represents our current task. This class allows the fit of predefined models with tuning of the possible important hyper-parameters variations, by fit_model() method. Finally, a prediction with the tuned and fitted model may be proceeded with the predict() method (check class documentation for further details).

In [18]:
# Current possible models
models = ["Linear Regression", "KNN", "Decision Tree", "SVM", "Random Forest", "Bagging"]

# Class instance
smo = SupervisedModelOptimization(pre_processed_data, data_output)
smo.summary()

No assigned test data, splitting the inputted train data
Input and output training Dimensions: (20088, 405) (20088,)
Input and output testing Dimensions: (8608, 405) (8608,)


##### Herewith, the class SupervisedModelOptimization splits the data automatically by a 0.3 proportion if no assigned test data. Accordingly, we have 200088 training samples and 8606 testing samples. 

In [19]:
model_scores = {}

for model in models:
  print(f"Running {model} prediction...")
  # Fit the model
  smo.fit_model(method=model)
  
  # Get the predictions
  predictions = smo.predict()
  
  # Get the score
  score = smo.score()
  
  # Get the model summary
  summary = smo.summary()
  #print(summary)
  # Store in the global variable
  model_scores[model] = (predictions, score, summary)

Running Bagging prediction...


KeyboardInterrupt: 

## WHEN MODEL FINISH RUNNING DO THE ANALYSIS